In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")
%matplotlib inline

from datetime import datetime
import time
import csv
sns.set_style("darkgrid")
results_2018 = pd.read_csv('./Runners_all_results_2018.csv', header = 0, encoding='latin-1')
results_2018['OfficialTime'] = results_2018['OfficialTime'].apply(lambda x: datetime.combine(datetime.min, datetime.strptime(x, '%H:%M:%S').time()) - datetime.min)
results_2019 = pd.read_csv('./Runners_all_results_2019.csv', header = 0, encoding='latin-1', error_bad_lines=False)
results_2019['OfficialTime'] = results_2019['OfficialTime'].apply(lambda x: datetime.combine(datetime.min, datetime.strptime(x, '%H:%M:%S').time()) - datetime.min)
results_2019.head(2)

b'Skipping line 26144: expected 20 fields, saw 21\n'


,BibNumber,FullName,SortName,AgeOnRaceDay,Gender,City,StateAbbrev,StateName,Zip,CountryOfResAbbrev,CountryOfResName,CountryOfCtzAbbrev,CountryOfCtzName,OfficialTime,RankOverall,RankOverGender,RankOverDivision,EventGroup,SubGroupLabel,SubGroup
0,2,Lawrence Cherono,"Cherono, Lawrence",30,M,Eldoret,NaN,NaN,NaN,KEN,Kenya,KEN,Kenya,02:07:57,1,1,1,Runners,NaN,NaN
1,6,Lelisa Desisa,"Desisa, Lelisa",29,M,Ambo,NaN,NaN,NaN,ETH,Ethiopia,ETH,Ethiopia,02:07:59,2,2,2,Runners,NaN,NaN


In [2]:
results_2019.Gender[results_2019.Gender == 'M'] = 1
results_2019.Gender[results_2019.Gender == 'F'] = 0
results_2019.head(2)

/home/micha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/micha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,BibNumber,FullName,SortName,AgeOnRaceDay,Gender,City,StateAbbrev,StateName,Zip,CountryOfResAbbrev,CountryOfResName,CountryOfCtzAbbrev,CountryOfCtzName,OfficialTime,RankOverall,RankOverGender,RankOverDivision,EventGroup,SubGroupLabel,SubGroup
0,2,Lawrence Cherono,"Cherono, Lawrence",30,1,Eldoret,NaN,NaN,NaN,KEN,Kenya,KEN,Kenya,02:07:57,1,1,1,Runners,NaN,NaN
1,6,Lelisa Desisa,"Desisa, Lelisa",29,1,Ambo,NaN,NaN,NaN,ETH,Ethiopia,ETH,Ethiopia,02:07:59,2,2,2,Runners,NaN,NaN


In [3]:
import requests
import pandas as pd
import geocoder

# script for returning elevation from lat, long
def get_elevation(lat, long):
    query = ('https://elevation-api.io/api/elevation'
             f'?points=({lat},{long})')
    r = requests.get(query, headers = {'User-agent': 'your bot 0.1'})
    if  r.status_code == requests.codes.ok:
        rj = r.json()# json object, various ways you can extract value
        # one approach is to use pandas json functionality:
        elevation = pd.io.json.json_normalize(rj, 'elevations')['elevation'].values[0]
        return elevation
    else:
        print("response status code: {}".format(r.status_code))
        return

def get_elevation_from_name(name):
    g = geocoder.arcgis(name)
    # print(g.lat, g.lng)
    elevation = get_elevation(g.lat, g.lng)
    # print(elevation)
    return elevation

In [7]:
cities = results_2019[['City']]
cities.drop_duplicates(inplace = True)
cities['Elevation'] = cities.apply(lambda x: get_elevation_from_name(x['City']), axis=1)
cities.to_csv("cities.csv", encoding='utf-8', index=False)

/home/micha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/micha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


response status code: 400
response status code: 400


In [8]:
print("hello")



hello
